In [1]:
import pandas as pd
import numpy as np
import pmdarima as pm
import plotly.graph_objects as go

from statsmodels.tsa.statespace.sarimax import SARIMAX

# 1. Cargar y preparar el dataset

In [2]:
df = pd.read_csv('../Data/transformed/transformed_open_meteo.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.sort_index(inplace=True)
print("Dimensiones del dataset:", df.shape)

Dimensiones del dataset: (230796, 8)


# 2. Definir las variables a predecir (targets)

In [3]:
variables_objetivo = ['temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean', 'windspeed_10m_max', 'precipitation_sum']

# 3. Obtener todas las provincias

In [4]:
provincias = df['provincia'].unique()

# 4. Detección de Outliers

- Método IQR

In [5]:
def detectar_outliers(series, threshold=1.5):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    limite_inferior = q1 - threshold * iqr
    limite_superior = q3 + threshold * iqr
    return (series < limite_inferior) | (series > limite_superior)

# 5. Auto Arima 

In [6]:
def ajustar_modelo_auto_arima(serie, pasos_forecast=7, seasonal=True, m=7):
    modelo_auto = pm.auto_arima(serie, seasonal=seasonal, m=m,
                                trace=False, error_action='ignore', suppress_warnings=True)
    order = modelo_auto.order
    seasonal_order = modelo_auto.seasonal_order
    print(f"Mejores parámetros: order={order}, seasonal_order={seasonal_order}")
    
    modelo = SARIMAX(serie, order=order, seasonal_order=seasonal_order)
    resultado = modelo.fit(disp=False)
    
    pred = resultado.get_forecast(steps=pasos_forecast)
    df_pred = pred.summary_frame()
    
    return df_pred, order, seasonal_order

In [7]:
lista_predicciones = []
pasos_forecast = 7

# 6. Predicción

In [ ]:
# 2. Iterar sobre cada provincia y cada variable
for provincia in provincias:
    print(f"\nProcesando provincia: {provincia}")
    df_prov = df[df['provincia'] == provincia].copy()
    
    for var in variables_objetivo:
        print(f"\nProcesando variable: {var} en provincia: {provincia}")
        
        # Detectar outliers e imputar: reemplazar outliers por NaN y luego interpolar
        outliers = detectar_outliers(df_prov[var], threshold=3)
        df_prov.loc[outliers, var] = np.nan
        df_prov[var] = df_prov[var].interpolate(method='time')
        df_prov[var] = df_prov[var].fillna(method='bfill').fillna(method='ffill')
        
        # Validar que haya suficientes datos para train-test
        if len(df_prov) <= pasos_forecast:
            print(f"No hay suficientes datos para train-test en {provincia} - {var}")
            continue
        
        # 3. Dividir la serie en entrenamiento y test
        train = df_prov[var].iloc[:-pasos_forecast]
        test = df_prov[var].iloc[-pasos_forecast:]
        
        try:
            # Ajustar el modelo usando auto_arima sobre el conjunto de entrenamiento
            df_pred, order, seasonal_order = ajustar_modelo_auto_arima(
                train, pasos_forecast=pasos_forecast, seasonal=True, m=7)
            
            # Alinear el índice de la predicción con el de la serie de test
            df_pred.index = test.index
            
            # Preparar DataFrame con los resultados para guardar en CSV
            df_pred_reset = df_pred.reset_index().rename(columns={'index': 'forecast_date'})
            df_pred_reset['provincia'] = provincia
            df_pred_reset['variable'] = var
            df_pred_reset['actual'] = test.values
            lista_predicciones.append(df_pred_reset)
            
            # 4. Visualizar el resultado con Plotly (Train, Test y Forecast)
            fig = go.Figure()
            # Serie de entrenamiento
            fig.add_trace(go.Scatter(
                x=train.index, y=train,
                mode='lines',
                name='Entrenamiento'
            ))
            # Serie de test
            fig.add_trace(go.Scatter(
                x=test.index, y=test,
                mode='lines',
                name='Test'
            ))
            # Predicción (forecast)
            fig.add_trace(go.Scatter(
                x=df_pred.index, y=df_pred['mean'],
                mode='lines',
                name='Predicción'
            ))
            # Intervalo de confianza
            fig.add_trace(go.Scatter(
                x=np.concatenate([df_pred.index, df_pred.index[::-1]]),
                y=np.concatenate([df_pred['mean_ci_upper'], df_pred['mean_ci_lower'][::-1]]),
                fill='toself',
                fillcolor='rgba(128,128,128,0.2)',
                line=dict(color='rgba(255,255,255,0)'),
                hoverinfo='skip',
                name='Intervalo de confianza'
            ))
            fig.update_layout(
                title=f"Forecast para '{var}' en {provincia}",
                xaxis_title="Fecha",
                yaxis_title=var
            )
            fig.show()
            
        except Exception as e:
            print(f"Error al procesar la variable {var} en {provincia}: {e}")



Procesando provincia: A Coruña


KeyError: 'province'

In [ ]:
# 5. Guardar todas las predicciones en un único CSV
if lista_predicciones:
    df_todas_pred = pd.concat(lista_predicciones, ignore_index=True)
    columnas_ordenadas = ['province', 'variable', 'forecast_date', 'mean', 'mean_se', 'mean_ci_lower', 'mean_ci_upper', 'actual']
    df_todas_pred = df_todas_pred[columnas_ordenadas]
    
    df_todas_pred.to_csv('predicciones_train_test.csv', index=False)
    print("\nArchivo CSV 'predicciones_train_test.csv' guardado correctamente.")
else:
    print("No se generaron predicciones.")